In [1]:
import json
import datetime as dt
import pandas as pd
import numpy as np
from sodapy import Socrata
import seaborn as sns

In [2]:
# Import the tokens to access the API
tokens = json.loads(open("./secret/tokens.json", "r").read())
app_token = tokens["app_token"]
secret_token = tokens["secret_token"]

In [3]:
def pull_data(source_url: str, identifier: str):
    """
    Pulls data using the Socrata API.

    Parameters:
        source_url (str): The URL of the source.
        identifier (str): The identifier of the dataset.
    """
    # Calling Socrata
    client = Socrata(source_url, app_token=app_token)
    client.timeout = 60

    # Save Records
    records = []
    offset = 0
    size = 10**5
    total_records = int(client.get(dataset_identifier=identifier, select="COUNT(*)")[0]["COUNT"])
    while len(records) != total_records:
        records.extend(client.get(dataset_identifier=identifier, offset=offset, limit=size))
        offset += size
    return (records, total_records)

In [4]:
# Pull data
source_url = "data.cityofnewyork.us"
identifier = "iebs-5yhr"
data, total_records = pull_data(source_url, identifier)

In [5]:
# Double Checking
print(len(data) == total_records)

True


In [6]:
# Store in DataFrame
df = pd.DataFrame(data)
df.head()

,report_category,geographic_subdivision,grade,year,category,number_tested,mean_scale_score,level_1,level_1_1,level_2,level_2_1,level_3,level_3_1,level_4,level_4_1,level_3_4,level_3_4_1,school_name
0,Citywide,Citywide,3,2023,All Students,49453,447,14225,28.8,11486,23.2,14154,28.6,9588,19.4,23742,48.0,NaN
1,Citywide,Citywide,4,2023,All Students,51008,450,10679,20.9,13530,26.5,14788,29.0,12011,23.5,26799,52.5,NaN
2,Citywide,Citywide,5,2023,All Students,53235,449,12486,23.5,14190,26.7,16717,31.4,9842,18.5,26559,49.9,NaN
3,Citywide,Citywide,6,2023,All Students,51996,447,13237,25.5,13882,26.7,13709,26.4,11168,21.5,24877,47.8,NaN
4,Citywide,Citywide,7,2023,All Students,53472,450,12976,24.3,12860,24.0,15838,29.6,11798,22.1,27636,51.7,NaN


In [7]:
df.report_category.value_counts()

report_category
School      588222
District     31525
Borough       4949
Citywide      1766
Name: count, dtype: int64

In [8]:
# Subset to only Borough and School Name
df = df[(df.report_category == "School") & (df.grade != "All Grades") & (df.mean_scale_score != "s")].reset_index(drop=True)
df.drop(columns="report_category", inplace=True)
df.head()

,geographic_subdivision,grade,year,category,number_tested,mean_scale_score,level_1,level_1_1,level_2,level_2_1,level_3,level_3_1,level_4,level_4_1,level_3_4,level_3_4_1,school_name
0,01M015,3,2023,All Students,24,455,4,16.7,5,20.8,11,45.8,4,16.7,15,62.5,P.S. 015 ROBERTO CLEMENTE
1,01M015,4,2023,All Students,17,454,1,5.9,6,35.3,8,47.1,2,11.8,10,58.8,P.S. 015 ROBERTO CLEMENTE
2,01M015,5,2023,All Students,30,441,10,33.3,11,36.7,7,23.3,2,6.7,9,30.0,P.S. 015 ROBERTO CLEMENTE
3,01M015,3,2022,All Students,21,594,4,19.0,12,57.1,4,19.0,1,4.8,5,23.8,P.S. 015 ROBERTO CLEMENTE
4,01M015,4,2022,All Students,30,596,6,20.0,14,46.7,5,16.7,5,16.7,10,33.3,P.S. 015 ROBERTO CLEMENTE


In [9]:
# Change the column names
new_columns = {
    "report_category": "category_type",
    "geographic_subdivision": "school_code",
    "category": "student_type",
    "mean_scale_score": "avg_students_tested",
    "level_1": "lvl_one_count",
    "level_1_1": "lvl_one_pct",
    "level_2": "lvl_two_count",
    "level_2_1": "lvl_two_pct",
    "level_3": "lvl_three_count",
    "level_3_1": "lvl_three_pct",
    "level_4": "lvl_four_count",
    "level_4_1": "lvl_four_pct",
    "level_3_4": "lvl_three_and_four_count",
    "level_3_4_1": "lvl_three_and_four_pct"
}

df.rename(columns=new_columns, inplace=True)

df.head()

,school_code,grade,year,student_type,number_tested,avg_students_tested,lvl_one_count,lvl_one_pct,lvl_two_count,lvl_two_pct,lvl_three_count,lvl_three_pct,lvl_four_count,lvl_four_pct,lvl_three_and_four_count,lvl_three_and_four_pct,school_name
0,01M015,3,2023,All Students,24,455,4,16.7,5,20.8,11,45.8,4,16.7,15,62.5,P.S. 015 ROBERTO CLEMENTE
1,01M015,4,2023,All Students,17,454,1,5.9,6,35.3,8,47.1,2,11.8,10,58.8,P.S. 015 ROBERTO CLEMENTE
2,01M015,5,2023,All Students,30,441,10,33.3,11,36.7,7,23.3,2,6.7,9,30.0,P.S. 015 ROBERTO CLEMENTE
3,01M015,3,2022,All Students,21,594,4,19.0,12,57.1,4,19.0,1,4.8,5,23.8,P.S. 015 ROBERTO CLEMENTE
4,01M015,4,2022,All Students,30,596,6,20.0,14,46.7,5,16.7,5,16.7,10,33.3,P.S. 015 ROBERTO CLEMENTE


In [10]:
# Converting integer values
df["grade"] = df["grade"].astype("int")
df["number_tested"] = df["number_tested"].astype("int")
df["avg_students_tested"] = df["avg_students_tested"].astype("int")
df["lvl_one_count"] = df["lvl_one_count"].astype("int")
df["lvl_two_count"] = df["lvl_two_count"].astype("int")
df["lvl_three_count"] = df["lvl_three_count"].astype("int")
df["lvl_four_count"] = df["lvl_four_count"].astype("int")
df["lvl_three_and_four_count"] = df["lvl_three_and_four_count"].astype("int")

# Converting percent values
df["lvl_one_pct"] = df["lvl_one_pct"].astype("float")
df["lvl_two_pct"] = df["lvl_two_pct"].astype("float")
df["lvl_three_pct"] = df["lvl_three_pct"].astype("float")
df["lvl_four_pct"] = df["lvl_four_pct"].astype("float")
df["lvl_three_and_four_pct"] = df["lvl_three_and_four_pct"].astype("float")

# Fix the year column
df["year"] = pd.to_datetime(df["year"], format='%Y')

In [11]:
# Dataframe Information
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 321359 entries, 0 to 321358
Data columns (total 17 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   school_code               321359 non-null  object        
 1   grade                     321359 non-null  int32         
 2   year                      321359 non-null  datetime64[ns]
 3   student_type              321359 non-null  object        
 4   number_tested             321359 non-null  int32         
 5   avg_students_tested       321359 non-null  int32         
 6   lvl_one_count             321359 non-null  int32         
 7   lvl_one_pct               321359 non-null  float64       
 8   lvl_two_count             321359 non-null  int32         
 9   lvl_two_pct               321359 non-null  float64       
 10  lvl_three_count           321359 non-null  int32         
 11  lvl_three_pct             321359 non-null  float64       
 12  lv

In [12]:
def get_school_borough(code: str):
    """
    Takes a code and returns the borough.
    """
    district = int(code[:2])
    if district in range(1, 7):
        return "Manhattan"
    elif district in range(7, 13):
        return "Bronx"
    elif district in range(13, 24) or district == 32:
        return "Brooklyn"
    elif district in range(24, 31):
        return "Queens"
    else:
        return "Staten Island"

In [13]:
# Get the Boroughs
df["borough"] = df["school_code"].apply(get_school_borough)
df.head()

,school_code,grade,year,student_type,number_tested,avg_students_tested,lvl_one_count,lvl_one_pct,lvl_two_count,lvl_two_pct,lvl_three_count,lvl_three_pct,lvl_four_count,lvl_four_pct,lvl_three_and_four_count,lvl_three_and_four_pct,school_name,borough
0,01M015,3,2023-01-01,All Students,24,455,4,16.7,5,20.8,11,45.8,4,16.7,15,62.5,P.S. 015 ROBERTO CLEMENTE,Manhattan
1,01M015,4,2023-01-01,All Students,17,454,1,5.9,6,35.3,8,47.1,2,11.8,10,58.8,P.S. 015 ROBERTO CLEMENTE,Manhattan
2,01M015,5,2023-01-01,All Students,30,441,10,33.3,11,36.7,7,23.3,2,6.7,9,30.0,P.S. 015 ROBERTO CLEMENTE,Manhattan
3,01M015,3,2022-01-01,All Students,21,594,4,19.0,12,57.1,4,19.0,1,4.8,5,23.8,P.S. 015 ROBERTO CLEMENTE,Manhattan
4,01M015,4,2022-01-01,All Students,30,596,6,20.0,14,46.7,5,16.7,5,16.7,10,33.3,P.S. 015 ROBERTO CLEMENTE,Manhattan


In [14]:
df.describe()

,grade,year,number_tested,avg_students_tested,lvl_one_count,lvl_one_pct,lvl_two_count,lvl_two_pct,lvl_three_count,lvl_three_pct,lvl_four_count,lvl_four_pct,lvl_three_and_four_count,lvl_three_and_four_pct
count,321359.000000,321359,321359.000000,321359.000000,321359.000000,321359.000000,321359.000000,321359.000000,321359.000000,321359.000000,321359.000000,321359.000000,321359.000000,321359.000000
mean,5.182929,2017-08-02 07:55:50.320358400,55.633332,420.691731,14.356838,29.228212,18.165765,32.714946,14.328732,24.605894,8.781998,13.453808,23.110730,38.059069
min,3.000000,2013-01-01 00:00:00,6.000000,193.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,4.000000,2015-01-01 00:00:00,19.000000,297.000000,3.000000,11.400000,5.000000,23.500000,3.000000,13.200000,0.000000,0.000000,5.000000,17.200000
50%,5.000000,2017-01-01 00:00:00,39.000000,329.000000,9.000000,25.000000,12.000000,33.300000,9.000000,23.600000,3.000000,7.700000,12.000000,34.800000
75%,7.000000,2019-01-01 00:00:00,70.000000,593.000000,19.000000,42.900000,23.000000,41.900000,18.000000,34.100000,9.000000,19.600000,29.000000,56.200000
max,8.000000,2023-01-01 00:00:00,804.000000,650.000000,365.000000,100.000000,359.000000,100.000000,250.000000,100.000000,421.000000,100.000000,494.000000,100.000000
std,1.675032,NaN,59.452884,139.769309,17.634635,22.124767,21.346847,14.026425,18.291297,15.402139,18.287192,16.317612,33.433618,25.285912


In [15]:
# Match the coordinates
temp = pd.read_csv("./data/school_info.csv")
temp.head()

,fiscal_year,system_code,location_code,location_name,BEDS,Managed_by_name,location_type_description,Location_Category_Description,Grades_text,Grades_final_text,...,Administrative_District_Name,community_school_sup_name,Tier_3_Support_Location_Name,Tier_3_Support_Leader_Name,Tier_2_Support_Location_Name,HighSchool_Network_Location_Code,HighSchool_Network_Name,HighSchool_Network_Superintendent,Community_district 1,Police_precinct
0,2020,15K001,K001,P.S. 001 The Bergen,331500010001,DOE,General Academic,Elementary,"PK,0K,01,02,03,04,05,SE","PK,0K,01,02,03,04,05",...,COMMUNITY SCHOOL DISTRICT 15,"SKOP, ANITA",NYCDOE Borough Office - Brooklyn North,NaN,School Support Team 5- Brooklyn North,NaN,NaN,NaN,307.0,72.0
1,2020,17K002,K002,Parkside Preparatory Academy,331700010002,DOE,General Academic,Junior High-Intermediate-Middle,"06,07,08,SE","06,07,08",...,COMMUNITY SCHOOL DISTRICT 17,"ELLIS, CLARENCE",NYCDOE Borough Office - Brooklyn South,Mauriciere de Govia,School Support Team 2- Brooklyn South,NaN,NaN,NaN,309.0,71.0
2,2020,13K003,K003,P.S. 003 The Bedford Village,331300010003,DOE,General Academic,Elementary,"PK,0K,01,02,03,04,05,SE","PK,0K,01,02,03,04,05",...,COMMUNITY SCHOOL DISTRICT 13,"SAMUELS, KAMAR",NYCDOE Borough Office - Brooklyn North,NaN,School Support Team 3- Brooklyn North,NaN,NaN,NaN,303.0,79.0
3,2020,75K004,K004,P.S. K004,307500013004,DOE,Special Education,Elementary,"PK,0K,01,02,03,04,05,SE","PK,0K,01,02,03,04,05,06,07,SE",...,CITYWIDE SPECIAL EDUCATION,"LOUISSAINT, KETLER",D75 CITYWIDE BCO,Tillman Roberto,Children First Network 752,NaN,NaN,NaN,305.0,75.0
4,2020,16K005,K005,P.S. 005 Dr. Ronald McNair,331600010005,DOE,General Academic,Elementary,"PK,0K,01,02,03,04,05,SE","PK,0K,01,02,03,04,05",...,COMMUNITY SCHOOL DISTRICT 16,"MARTIN, YOLANDA",NYCDOE Borough Office - Brooklyn North,NaN,School Support Team 4- Brooklyn North,NaN,NaN,NaN,303.0,81.0


In [16]:
# Only interested in longitude and latitude
temp.columns = temp.columns.str.lower()
temp.columns

Index(['fiscal_year', 'system_code', 'location_code', 'location_name', 'beds',
       'managed_by_name', 'location_type_description',
       'location_category_description', 'grades_text', 'grades_final_text',
       'open_date', 'status_descriptions', 'primary_building_code',
       'primary_address_line_1', 'state_code', 'x_coordinate', 'y_coordinate',
       'longitude', 'latitude', 'community_district', 'council-district',
       'census_tract', 'borough_block_lot', 'nta', 'nta_name',
       'principal_name', 'principal_title', 'principal_phone_number',
       'fax_number', 'geographical_district_code',
       'administrative_district_code', 'administrative_district_name',
       'community_school_sup_name', 'tier_3_support_location_name',
       'tier_3_support_leader_name', 'tier_2_support_location_name',
       'highschool_network_location_code', 'highschool_network_name',
       'highschool_network_superintendent', 'community_district 1',
       'police_precinct'],
      dtype=

In [17]:
# Merge the data by school code

df_merged = df.merge(temp[["system_code", "longitude", "latitude"]],
         how="left",
         left_on="school_code",
         right_on="system_code")
df_merged.drop(columns="system_code", inplace=True)
df_merged.head()

,school_code,grade,year,student_type,number_tested,avg_students_tested,lvl_one_count,lvl_one_pct,lvl_two_count,lvl_two_pct,lvl_three_count,lvl_three_pct,lvl_four_count,lvl_four_pct,lvl_three_and_four_count,lvl_three_and_four_pct,school_name,borough,longitude,latitude
0,01M015,3,2023-01-01,All Students,24,455,4,16.7,5,20.8,11,45.8,4,16.7,15,62.5,P.S. 015 ROBERTO CLEMENTE,Manhattan,-73.978747,40.722075
1,01M015,4,2023-01-01,All Students,17,454,1,5.9,6,35.3,8,47.1,2,11.8,10,58.8,P.S. 015 ROBERTO CLEMENTE,Manhattan,-73.978747,40.722075
2,01M015,5,2023-01-01,All Students,30,441,10,33.3,11,36.7,7,23.3,2,6.7,9,30.0,P.S. 015 ROBERTO CLEMENTE,Manhattan,-73.978747,40.722075
3,01M015,3,2022-01-01,All Students,21,594,4,19.0,12,57.1,4,19.0,1,4.8,5,23.8,P.S. 015 ROBERTO CLEMENTE,Manhattan,-73.978747,40.722075
4,01M015,4,2022-01-01,All Students,30,596,6,20.0,14,46.7,5,16.7,5,16.7,10,33.3,P.S. 015 ROBERTO CLEMENTE,Manhattan,-73.978747,40.722075


In [18]:
# Check for NULL values
df_merged[pd.isnull(df_merged["longitude"])].school_code.unique()

array(['05M371', '19K935', '20K936', '24Q419', '31R085'], dtype=object)

In [19]:
# Locate these schools manually
school_code_match = [
    {"school_code": "05M371", "longitude": -73.952170, "latitude": 40.814970},
    {"school_code": "19K935", "longitude": -73.876400, "latitude": 40.681190},
    {"school_code": "20K936", "longitude": -74.020290, "latitude": 40.642790},
    {"school_code": "24Q419", "longitude": -73.858920, "latitude": 40.759470},
    {"school_code": "31R085", "longitude": -74.0850979, "latitude": 40.6183021},
]
# Match
for school_code in school_code_match:
    condition = df_merged["school_code"] == school_code["school_code"]
    df_merged['longitude'] = df_merged['longitude'].fillna(condition.map({True:school_code["longitude"]}))
    df_merged['latitude'] = df_merged['latitude'].fillna(condition.map({True:school_code["latitude"]}))

In [20]:
# Check for NULL values again
df_merged[pd.isnull(df_merged["longitude"])].school_code.unique()

array([], dtype=object)

In [21]:
# Export the data
df_merged.to_excel("data/nyc_schools_ela_result.xlsx", index=False)